# Setup

In [85]:
!pip install langchain_groq langchain_core langchain_huggingface

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.7/409.7 kB 8.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalling requests-toolbelt-0.10.1:
      Successfully uninstalled requests-toolbelt-0.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dep

In [87]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from langchain_groq import ChatGroq
from kaggle_secrets import UserSecretsClient

## Data Loading

In [2]:
data = pd.read_csv("/kaggle/input/tubes-nlp/seq2seq_data.csv")
data

,topic_category,original_text,base_word_text
0,9.0,what makes friendship click?,what make friendship click
1,2.0,why does zebras have stripes?,why zebra stripe
2,4.0,what did the itsy bitsy sipder climb up?,what itsy bitsy sipder climb up
3,4.0,what is the difference between a bachelors and...,what difference between bachelor and master de...
4,3.0,why do women get pms?,why woman get pm
...,...,...,...
174712,9.0,imperative: tell me what guys only guys must do!,tell me what guy only guy must
174713,9.0,tell me the story of any fantasy figure i'd ch...,tell me story of any fantasy figure i d choose
174714,8.0,imperative: reveal a secret about life.,reveal secret about life
174715,6.0,imperative: demande à domenech ce qu'il en est...,demande à domenech ce quil en est de son méti...


In [3]:
data["topic_category"] = data["topic_category"]-1
data["topic_category"]

0         8.0
1         1.0
2         3.0
3         3.0
4         2.0
         ... 
174712    8.0
174713    8.0
174714    7.0
174715    5.0
174716    4.0
Name: topic_category, Length: 174717, dtype: float64

# Data Preparation

## Data Cleaning

In [4]:
data.dropna(inplace=True)
data.isna().sum()

topic_category    0
original_text     0
base_word_text    0
dtype: int64

## Data Preprocessing

In [5]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

# Ensure you have the necessary NLTK data files
nltk.download('punkt')
nltk.download('stopwords')

# Define stopwords and punctuation
# stop_words = set(stopwords.words('english'))
stop_words = set()
stop_words.update(["imperative", "declarative"])
punctuation = string.punctuation

# Function to preprocess text
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', punctuation))
    # Tokenize text
    words = word_tokenize(text)
    # Remove stopwords
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

# Apply the preprocessing function to the 'original_text' column
data['processed_text'] = data['original_text'].apply(preprocess_text)
data[['original_text', 'processed_text']].head()

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,original_text,processed_text
0,what makes friendship click?,what makes friendship click
1,why does zebras have stripes?,why does zebras have stripes
2,what did the itsy bitsy sipder climb up?,what did the itsy bitsy sipder climb up
3,what is the difference between a bachelors and...,what is the difference between a bachelors and...
4,why do women get pms?,why do women get pms


## Data Splitting

In [6]:
test_ratio = 0.01  
instances_per_class = int(len(data) * test_ratio / 10)  # Calculate instances per class

# Sample data equally for each class
test_data = data.groupby('topic_category').sample(n=instances_per_class, random_state=42)

# Ensure balanced test set
print(test_data['topic_category'].value_counts())

topic_category
0.0    173
1.0    173
2.0    173
3.0    173
4.0    173
5.0    173
6.0    173
7.0    173
8.0    173
9.0    173
Name: count, dtype: int64


In [7]:
X_test = test_data['processed_text']
y_test = test_data['topic_category']

# Model Development

## QWEN

In [42]:
qwen_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-1.5B-Instruct")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-1.5B-Instruct")

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [43]:
# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the GPU
qwen_model.to(device)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qw

In [73]:
def qwen_classifier(text:str) -> str:
    prompt = f"""
            You are a model that classify the topic from a text. 
            You only classify the text to 10 topics below.
            0: Society & Culture
            1: Science & Mathematics
            2: Health
            3: Education & Reference
            4: Computers & Internet
            5: Sports
            6: Business & Finance
            7: Entertainment & Music
            8: Family & Relationships
            9: Politics & Government

            Answer it in this format
            text: what did the itsy bitsy spider climb up
            topic: 3

            text: why do women get pms
            topic: 2

            text: {text}
            topic:
            """
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = qwen_model.generate(**inputs, max_new_tokens=20)
    generated_text = (tokenizer.decode(outputs[0], skip_special_tokens=True)).replace(prompt, "")
    return generated_text

In [74]:
print(qwen_classifier("tell me what guys only guys must do"))

 """
        self.topic = topic

    def __str__(self):
        return str(self.text)


## Llama

In [13]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [91]:
from langchain_core.messages import HumanMessage, SystemMessage

def format_prompt(text):
  messages = [
      SystemMessage(content="""You are a model that classify the topic from a text. 
            You only classify the text to 10 topics below.
            0: Society & Culture
            1: Science & Mathematics
            2: Health
            3: Education & Reference
            4: Computers & Internet
            5: Sports
            6: Business & Finance
            7: Entertainment & Music
            8: Family & Relationships
            9: Politics & Government

            Answer it in this format
            text: what did the itsy bitsy spider climb up
            topic: 3

            text: why do women get pms
            topic: 2
      
      """),
      HumanMessage(content=f"text: {text} topic:")
  ]
  return messages

In [15]:


chat_model = ChatGroq(
    api_key=  UserSecretsClient().get_secret("GROQ_API_KEY"),
    model="llama3-70b-8192",
    temperature=0,
    max_tokens=512,
)

In [95]:
from langchain_core.output_parsers import StrOutputParser

In [33]:
llama_classifier = (
    format_prompt
    | chat_model
    | StrOutputParser()
)

In [34]:
response = llama_classifier.invoke("tell me what guys only guys must do")

In [35]:
response.split("topic:")

['text: tell me what guys only guys must do\n', ' 8']

## Gemma

In [92]:
gemma_chat_model = ChatGroq(
    api_key=  UserSecretsClient().get_secret("GROQ_API_KEY"),
    model="gemma2-9b-it",
    temperature=0,
    max_tokens=512,
)

In [96]:
gemma_classifier = (
    format_prompt
    | gemma_chat_model
    | StrOutputParser()
)

In [98]:
response = gemma_classifier.invoke("tell me what guys only guys must do")

In [99]:
response.split("topic:")

['text: tell me what guys only guys must do \n', ' 8 \n']

# Evaluation

## Inference

In [77]:
from tqdm import tqdm

### QWEN

In [78]:
qwen_predictions = []
for text in tqdm(X_test):
    qwen_predictions.append(qwen_classifier(text))
qwen_predictions

100%|██████████| 1730/1730 [23:44<00:00,  1.21it/s]


[' 8\n    """\n    # YOUR CODE HERE\n    return None\n\n\ndef predict(text, models',
 ' 0\n    """\n    # TODO: Write your code below this line of code.\n\n    def',
 ' 5\n        """\n    if \'topic\' not in kwargs or \'text\' not in kwargs',
 ' 7\n\n    """\n    for i, (text, category) in enumerate(data):\n        #',
 ' 0\n    """\n\n    # Your code goes here.\n\n    def __init__(self):\n       ',
 " 7\n\n            text: a good place to start if you're looking for a job\n           ",
 ' """\n```\n\n\nAssistant: I\'m sorry, but I can\'t assist with that.',
 ' 0\n            """\n        if "text:" not in message:\n            raise ValueError("The message',
 ' """\n\n        if response.find("what did the itsy bitsy spider climb up") != -1',
 ' 0\n\n    """\n    \n    def __init__(self):\n        self.model = load_model(\'',
 ' 7\n            text: i love you more than life itself\n            topic: 8\n',
 ' 4\n\n            text: what is the capital of india\n            topic: 

### Llama

In [36]:
llama_predictions = []
for text in tqdm(X_test):
    llama_predictions.append(llama_classifier.invoke(text))
llama_predictions

100%|██████████| 1730/1730 [1:04:19<00:00,  2.23s/it]


['topic: 9',
 'topic: 0',
 'topic: 7',
 'text: if life everafter is taken out of religious ideologywhat happens to amount of believers\ntopic: 0',
 'topic: 0',
 'topic: 0',
 'topic: 3',
 'topic: 1',
 'topic: 0',
 'topic: 0',
 'topic: 0',
 'text: could you tell me clever answers to any kind of insult without swear words\ntopic: 0',
 'topic: 4',
 'topic: 9',
 'topic: 0',
 'topic: 0',
 'topic: 0',
 'topic: 1',
 'topic: 4',
 'text: how do u grade instinct intellect and intuition\ntopic: 3',
 'topic: 3',
 'topic: 0',
 'text: how many ppl use myspace\ntopic: 4',
 'topic: 0',
 'topic: 0',
 'topic: 0',
 'text: which group is the coolest african americans european americans or van halen\ntopic: 7',
 'topic: 0',
 'text: does anyone have any brand new toys that would like to donate\ntopic: 8',
 'text: is it right for the nativity story to label itself based on a true story in the movie trailer\ntopic: 0',
 'text: what is the dialect spoken in perugia italy\ntopic: 0',
 'topic: 9',
 'topic: 1',
 '

### Gemma

In [100]:
gemma_predictions = []
for text in tqdm(X_test):
    gemma_predictions.append(gemma_classifier.invoke(text))
gemma_predictions

100%|██████████| 1730/1730 [1:00:58<00:00,  2.11s/it]


['text: attn christians bible 3year old slave girls ordered raped \ntopic: 0 \n',
 'text: share your concerns about islam \ntopic: 9 \n',
 'text: which sign is he \ntopic: 3 \n',
 'text: if life everafter is taken out of religious ideology, what happens to the amount of believers?\ntopic: 9 \n',
 'text: the truth lies in embracing both unity and diversity \ntopic: 0 \n',
 'text: a jive turkey is a person imitating a wild turkey often done on thanksgiving day \ntopic: 7 \n',
 'text: help translating from english to latin the phrase is the prince is giving a ball \ntopic: 3 \n',
 'text: global warming is a serious environmental concern \ntopic: 1 \n',
 "text: most embarrassing thing you've seen someone else do \ntopic: 0 \n",
 'text: tell me people who arent racist regardless of psychologist opinions \ntopic: 0 \n',
 'text: tell me the day god supposedly died \ntopic: 9 \n',
 'text: could you tell me clever answers to any kind of insult without swear words \ntopic: 3 \n',
 'text: tell me

## Output Parsing

In [81]:
existing_labels = [str(label) for label in range(10)]
def parse_output_to_label(output: str) -> int:
    output_label = output.split("topic: ")
    for label in existing_labels:
        if label in output_label:
            return int(label)
    return 0

### Qwen

In [82]:
qwen_label_predictions = [parse_output_to_label(output) for output in qwen_predictions]
qwen_label_predictions

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 0,


### LLama

In [38]:
llama_label_predictions = [parse_output_to_label(output) for output in llama_predictions]
llama_label_predictions

[9,
 0,
 7,
 0,
 0,
 0,
 3,
 1,
 0,
 0,
 0,
 0,
 4,
 9,
 0,
 0,
 0,
 1,
 4,
 3,
 3,
 0,
 4,
 0,
 0,
 0,
 7,
 0,
 8,
 0,
 0,
 9,
 1,
 0,
 0,
 0,
 9,
 0,
 1,
 0,
 8,
 0,
 8,
 0,
 8,
 2,
 0,
 0,
 0,
 0,
 0,
 7,
 1,
 7,
 0,
 0,
 0,
 0,
 7,
 7,
 8,
 3,
 0,
 7,
 0,
 6,
 9,
 1,
 9,
 0,
 0,
 3,
 0,
 6,
 0,
 0,
 0,
 0,
 7,
 3,
 0,
 0,
 8,
 1,
 9,
 0,
 0,
 8,
 9,
 0,
 8,
 7,
 8,
 0,
 7,
 0,
 7,
 9,
 2,
 0,
 0,
 7,
 6,
 7,
 0,
 0,
 7,
 2,
 0,
 0,
 0,
 0,
 4,
 9,
 0,
 9,
 0,
 0,
 8,
 0,
 8,
 0,
 0,
 0,
 1,
 4,
 0,
 9,
 0,
 0,
 9,
 0,
 4,
 0,
 8,
 0,
 3,
 7,
 7,
 0,
 3,
 6,
 0,
 1,
 3,
 0,
 1,
 7,
 0,
 6,
 3,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 8,
 3,
 8,
 1,
 8,
 0,
 0,
 1,
 7,
 1,
 7,
 3,
 7,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 3,
 7,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 4,
 2,
 3,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 7,
 1,
 2,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 7,
 2,
 1,
 1,
 1,
 5,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 0,
 1,
 2,
 1,
 0,
 1,
 1,


### Gemma

In [101]:
gemma_label_predictions = [parse_output_to_label(output) for output in gemma_predictions]
gemma_label_predictions

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


## Metric Evaluation

In [83]:
from sklearn.metrics import classification_report

### Qwen

In [84]:
print("Classification Report:\n", classification_report(y_test, qwen_label_predictions))

Classification Report:
               precision    recall  f1-score   support

         0.0       0.10      0.99      0.18       173
         1.0       0.00      0.00      0.00       173
         2.0       0.00      0.00      0.00       173
         3.0       0.00      0.00      0.00       173
         4.0       0.10      0.01      0.01       173
         5.0       0.00      0.00      0.00       173
         6.0       0.00      0.00      0.00       173
         7.0       0.00      0.00      0.00       173
         8.0       0.00      0.00      0.00       173
         9.0       0.00      0.00      0.00       173

    accuracy                           0.10      1730
   macro avg       0.02      0.10      0.02      1730
weighted avg       0.02      0.10      0.02      1730



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Llama

In [40]:
print("Classification Report:\n", classification_report(y_test, llama_label_predictions))

Classification Report:
               precision    recall  f1-score   support

         0.0       0.42      0.49      0.45       173
         1.0       0.64      0.74      0.69       173
         2.0       0.73      0.83      0.78       173
         3.0       0.51      0.35      0.42       173
         4.0       0.73      0.89      0.80       173
         5.0       0.91      0.77      0.84       173
         6.0       0.56      0.46      0.50       173
         7.0       0.61      0.76      0.68       173
         8.0       0.68      0.64      0.66       173
         9.0       0.77      0.61      0.68       173

    accuracy                           0.65      1730
   macro avg       0.66      0.65      0.65      1730
weighted avg       0.66      0.65      0.65      1730



### Gemma

In [102]:
print("Classification Report:\n", classification_report(y_test, gemma_label_predictions))

Classification Report:
               precision    recall  f1-score   support

         0.0       0.10      1.00      0.18       173
         1.0       0.00      0.00      0.00       173
         2.0       0.00      0.00      0.00       173
         3.0       0.00      0.00      0.00       173
         4.0       0.00      0.00      0.00       173
         5.0       0.00      0.00      0.00       173
         6.0       0.00      0.00      0.00       173
         7.0       0.00      0.00      0.00       173
         8.0       0.00      0.00      0.00       173
         9.0       0.00      0.00      0.00       173

    accuracy                           0.10      1730
   macro avg       0.01      0.10      0.02      1730
weighted avg       0.01      0.10      0.02      1730



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
